In [3]:
import numpy as np #Importation des packages et bibliothèques
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import f_oneway, kruskal
from sklearn.ensemble import IsolationForest

dataSet = sp.loadmat('breastw.mat') # On importe le dataset 
X = dataSet['X'] # On récupère les caractéristiques des points du data
y = dataSet['y'] # On récupère les labels des points du dataset

Contamination = [0.35] #Création des plages de valeurs des paramètres optimisés
nombre_estim =[30,30,40,50,10,20,30,30,30,40]
MS=[50,20,40,30,10,40,40,100,100,30]
plage_features = [2,6,5,2,9,4,4,4,7,6]
skf= StratifiedKFold(n_splits=10,shuffle=False,random_state=None) # On splits nos données en 10
for i in range(0,9) :  # boucle for pour changer de combinaison, 10 combinaisons 
        test=False
        for train_index, test_index in skf.split(X, y):
                X_train, X_test = X[train_index,:], X[test_index,:]  #Séparation des données en données d'entrainement et de test
                y_train, y_test = y[train_index], y[test_index]
                model = IsolationForest(contamination=Contamination[0], n_estimators=nombre_estim[i], max_samples=MS[i],max_features=plage_features[i]) #Paramétrage du modèle Isolation Forest
                model.fit(X_train)
                y_pred=model.predict(X_test) #Récupération des labels prédit de chaque points
                y_pred[y_pred>0]=0 #le modèle renvoie 1 et -1 comme label, conversion en 0 et 1 pour le calcul du F1 score
                y_pred[y_pred<0]=1
                F1=round(f1_score(y_test,y_pred),3) #Calcul du F1 score
                if test == False :
                        matrice = np.array([[F1]])  #Construction d'une matrice de stockage des F1 scores, réalisé qu'une fois grâce au boolean test
                        test = True
                else :
                        matrice = np.append(matrice,np.array([F1]).reshape(1,1),axis=0) #Ajout des F1 score à la matrice
        if i == 0  :
                df=pd.DataFrame(matrice,columns=['[{},{},{},{}]'.format(round(Contamination[0],3),nombre_estim[i],MS[i],plage_features[i])]) #Création de data frames pour contenir nos résultats
        else :
                df1=pd.DataFrame(matrice,columns=['[{},{},{},{}]'.format(round(Contamination[0],3),nombre_estim[i],MS[i],plage_features[i])])
                df=pd.concat([df, df1], axis = 1)
display(df) #Affichage des F1 scores des diffférentes combinaisons     

,"[0.35,30,50,2]","[0.35,30,20,6]","[0.35,40,40,5]","[0.35,50,30,2]","[0.35,10,10,9]","[0.35,20,40,4]","[0.35,30,40,4]","[0.35,30,100,4]","[0.35,30,100,7]"
0,0.920,0.894,0.880,0.894,0.913,0.870,0.875,0.894,0.894
1,0.939,0.894,0.898,0.939,0.917,0.894,0.898,0.920,0.923
2,0.920,0.920,0.941,0.941,0.902,0.920,0.898,0.920,0.939
3,0.898,0.894,0.894,0.894,0.913,0.913,0.870,0.894,0.889
4,0.980,0.936,0.917,0.958,0.960,0.980,0.917,0.870,0.980
5,0.844,0.936,0.958,0.917,0.478,0.894,0.939,0.917,0.917
6,0.936,0.958,0.958,0.933,0.870,0.939,0.913,0.957,0.936
7,0.979,0.979,0.979,0.979,0.939,0.980,0.979,1.000,1.000
8,0.958,0.979,0.979,0.957,0.979,0.958,0.958,0.958,0.957
9,1.000,1.000,0.980,1.000,0.979,0.980,1.000,1.000,0.980


In [4]:
from scipy.stats import f_oneway, kruskal
from scipy import stats
import scikit_posthocs as sp #Importation des packages et bibliothèques
t,p = kruskal(df['[0.35,30,50,2]'],df['[0.35,30,20,6]'],df['[0.35,40,40,5]'],df['[0.35,50,30,2]'],df['[0.35,10,10,9]'],df['[0.35,20,40,4]'],df['[0.35,30,40,4]'],df['[0.35,30,100,4]'],df['[0.35,30,100,7]'])
print('t statistic: %.3f' % t) #Affichage résultat tests statistiques
print('p value: %.3f' % p)    

sp.posthoc_conover([df['[0.35,30,50,2]'],df['[0.35,30,20,6]'],df['[0.35,40,40,5]'],df['[0.35,50,30,2]'],df['[0.35,10,10,9]'],df['[0.35,20,40,4]'],df['[0.35,30,40,4]'],df['[0.35,30,100,4]'],df['[0.35,30,100,7]']]) #Posthoc

t statistic: 2.400
p value: 0.966


,1,2,3,4,5,6,7,8,9
1,1.000000,0.797569,0.884856,0.934046,0.324149,0.706634,0.361712,0.611068,0.996698
2,0.797569,1.000000,0.911047,0.862043,0.464665,0.904489,0.511156,0.800763,0.800763
3,0.884856,0.911047,1.000000,0.950509,0.399638,0.816785,0.442348,0.715871,0.888124
4,0.934046,0.862043,0.950509,1.000000,0.366050,0.768980,0.406576,0.670137,0.937337
5,0.324149,0.464665,0.399638,0.366050,1.000000,0.540741,0.940628,0.631477,0.326165
6,0.706634,0.904489,0.816785,0.768980,0.540741,1.000000,0.590964,0.894664,0.709708
7,0.361712,0.511156,0.442348,0.406576,0.940628,0.590964,1.000000,0.685254,0.363877
8,0.611068,0.800763,0.715871,0.670137,0.631477,0.894664,0.685254,1.000000,0.613966
9,0.996698,0.800763,0.888124,0.937337,0.326165,0.709708,0.363877,0.613966,1.000000
